<a href="https://colab.research.google.com/github/ARM-02/FlexFit/blob/main/FlexFit_Final_Working_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import csv
import pandas as pd

#-------------------#
# Read CSV FIle
# Load Data
#-------------------#
from google.colab import files
uploaded = files.upload()
database = pd.read_csv("exercise_database.csv")


def load_exercise_database(csv_file_path):
    # Create an empty dictionary to hold all exercise categories
    exercise_database = {}

    # Open and read the CSV file
    with open(csv_file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Iterate through each row in the CSV file
        for row in reader:
            category = row['Category']
            exercise_name = row['Exercise Name']

            # Create a dictionary for this exercise
            exercise_data = {
                "description": row['Description'],
                "targeted_muscles": row['Targeted Muscles'].split(", "),  # Convert muscles to a list
                "equipment_needed": row['Equipment Needed'],
                "difficulty": row['Difficulty'],
                "repetitions": row['Repetitions']
            }

            # If the category doesn't exist in the database, create it
            if category not in exercise_database:
                exercise_database[category] = {}

            # Add the exercise to the corresponding category
            exercise_database[category][exercise_name] = exercise_data

    return exercise_database



#-------------------------------#
#
# Build Your Workout Modality
#
#-------------------------------#

#---------
# STEP 1: OBTAIN USER'S INPUT
#---------

def get_user_input():
  #FITNESS GOAL
  fitness_goal = int(input("""Indicate your fitness goal:
                            \n1 = Build Muscle,
                            \n2 = Improve Mobility,
                            \n3 = Improve Cardio \n """))

  #validate inputed goal
  while fitness_goal > 3 or fitness_goal < 1:
    print("\nPlease select a valid fitness goal")
    fitness_goal = int(input("""\nIndicate your fitness goal:
                            \n1 = Build Muscle,
                            \n2 = Improve Mobility,
                            \n3 = Improve Cardio \n """))

  #Store fitness goal in
  if fitness_goal == 1:
    fitness_goal = 'build_muscle'
  if fitness_goal == 2:
    fitness_goal = 'improve_mobility'
  if fitness_goal ==3:
    fitness_goal = 'improve_cardio'

  #DAYS X WEEK
  wk_days = int(input("\nHow many days per week do you want to workout?: "))

   #validate inputed number of days
  while wk_days > 7:
    print("\nWe're sorry, the maximum number of days per week is 7, please re-enter your desired number of days per week")
    wk_days = int(input("\nHow many days per week do you want to workout?: "))

  #WK DURATION
  wk_length = int(input("\nHow many minutes per day do you want to workout?: "))

  #validate inputed duration
  while wk_length > 60:
    print("\nWe're sorry, the maximum workout time is 60 minutes, please re-enter your desired workout time")
    wk_length = int(input("\nHow many minutes per day do you want to workout?: "))


  return {"fitness_goal": fitness_goal, "wk_days": wk_days, "wk_length": wk_length}



#---------
# STEP 2: CREATE EMPTY WORKOUT STRUCTURE BASED ON INPUT
#---------

def get_wk_structure(user_input):
  #Calculate the number of exercises needed per day
  #Adding 9 ensures that any remainder when dividing by 10 pushes the value up to the next multiple of 10.
  rounded_wk_length = round((user_input['wk_length'] + 9)//10) * 10
  number_of_exercises = int(rounded_wk_length/10)

  #Create the empty hashtable for the specific workout plan
  empty_wk_plan ={}

  #Create a key for each day
  for i in range(user_input['wk_days']):
    empty_wk_plan[f'Day {i+1}'] = {}


  #Create a value for each exercise
  for day in empty_wk_plan:
    for exercise in range(number_of_exercises):
      empty_wk_plan[day][f'Exercise {exercise+1}'] = {}


  #Return the empty workout plan
  return empty_wk_plan




#---------
# STEP 3: FILTER DATASET BASED ON USER'S FITNESS GOAL
#---------

def get_filtered_database(user_input):
# Extract the fitnes goal
  goal = user_input['fitness_goal']


#Load the main exerxise database
  database = load_exercise_database("exercise_database.csv")

# First define which exersises fit each goal
  build_muscle_guide = [
    "back_exercises",
    "bicep_exercises",
    "tricep_exercises",
    "shoulder_exercises",
    "leg_exercises",
    "glute_exercises",
    "hamstring_exercises",
]

  improve_mobility_guide = [
    "stretches",
    "core_exercises",
    "hamstring_exercises",
    "glute_exercises",
]

  improve_cardio_guide = [
    "cardio_exercises",
    "endurance_moves",
    "core_exercises",
]


# Now, match the fitness goal to the correct guide
  if goal == 'build_muscle':
    guide = build_muscle_guide
  elif goal == 'improve_mobility':
    guide = improve_mobility_guide
  elif goal == 'improve_cardio':
    guide = improve_cardio_guide


# Finally, filter the database based on the guide

  filtered_database = {key: database[key] for key in guide if key in database}


  return filtered_database
  # Retunrs a dictionary in which:
  # keys --> exercise categories (ex, "cardio_exercises")
  # values --> dictionaries of specific exercises.



#---------
# STEP 4: FILL THE PERSONALIZED WORKOUT PLAN
#---------

def fill_wk_plan(user_input, wk_structure):

# Get the filtered database
  database = get_filtered_database(user_input)

#Get the empty workout plan
  empty_wk_plan = get_wk_structure(user_input)

# Create a new dictionary to store the wk plan. This way users can look them up later.
  filled_workout_plan = {}

# Initialize a set to keep track of already selected exercises for each day
  used_exercises = set()


# Fill the empty wk plan with exercises
  for day in empty_wk_plan:
    filled_workout_plan[day] = {}  # Initialize each day in the workout plan
    for exercise in empty_wk_plan[day]:
      #Select a random muscle group (just the key)
      random_muscle_group = random.choice(list(database.keys()))

      # Get the list of exercises for the selected muscle group
      available_exercises = list(database[random_muscle_group].items())

      # Filter out exercises that have already been used
      available_exercises = [exercise for exercise in available_exercises if exercise[0] not in used_exercises]

     # If there are no more available exercises, break the loop (or handle accordingly)
      if not available_exercises:
        print("No more unique exercises left to add. Please adjust your workout plan.")
        break

      # Select a random exercise from the filtered list of available exercises
      random_exercise = random.choice(available_exercises)

      # Mark the selected exercise as used
      used_exercises.add(random_exercise[0])

      # Add the exercise to the filled workout plan
      filled_workout_plan[day][exercise] = {
        "name": random_exercise[0],  # Exercise name
        "details": random_exercise[1]  # Exercise details dictionary
            }


  return filled_workout_plan


#---------
# STEP 5: DISPLAY THE PERSONALIZED WORKOUT PLAN
#---------

def display_filled_wk_plan(fill_wk_plan):
  print('Repeat the routine 3 times to meet youre  desiref workout time')
  for day,exercises in fill_wk_plan.items(): #iterates over each day and each exercise (of each day)
    print(f'{day}:') #print each day

    for exercise, details in exercises.items():
      exercise_name = details["name"]
      exercise_info = details["details"]
      print(f"""\n  - {exercise_name}:
                  {exercise_info['description']}
                  Repetitions: {exercise_info['repetitions']}
                  Difficulty: {exercise_info['difficulty']}
              """)

#---------
# STEP 6: SAVE THE WORKOUT PLAN
#---------
def save_workout_plan(filled_workout_plan):
    """
    Save the entire workout plan under a user-defined name in the global saved_workouts.
    """
    global saved_workouts  # Access the global dictionary


    # Ask the user to provide a name for the workout
    workout_name = input("Enter a name for your workout plan: ").strip()

    # Check if the name already exists
    if workout_name in saved_workouts:
        overwrite = input(f"A workout named '{workout_name}' already exists. Overwrite? (yes/no): ").strip().lower()
        if overwrite != 'yes':
            print("Workout not saved.")
            return

    # Save the workout plan under the specified name
    saved_workouts[workout_name] = filled_workout_plan
    print(f"Workout '{workout_name}' has been saved successfully!")


#---------
# STEP 7: BUILD YOUR WORKOUT MODALITY - MAIN FUNCTION
#---------

def build_your_workout():

  # Get the user's input
  user_input = get_user_input()

  # Generate the empty workout structure
  wk_structure = get_wk_structure(user_input)

  # Filter database based on fitness goal
  filtered_database = get_filtered_database(user_input)

  # Fill workout plan
  filled_wk_plan = fill_wk_plan(user_input, wk_structure)

  #Diplay workout plan
  display_filled_wk_plan(filled_wk_plan)

  #Save the workout
  saved_workout = save_workout_plan(filled_wk_plan)





#---------------------------------#
#
# Browse Your Workouts Modality
#
#---------------------------------#

def display_saved_workout():
    global saved_workouts  # Access the global dictionary

    if not saved_workouts:
        print("\nNo saved workouts found.")
        return

    print("\nYour Saved Workouts:")
    for workout_name in saved_workouts.keys():
        print(f"- {workout_name}")

    # Allow the user to select a workout to view
    selected_workout = input("\nEnter the name of the workout you want to view: ").strip()
    if selected_workout in saved_workouts:
        print(f"\nWorkout: {selected_workout}")
        display_filled_wk_plan(saved_workouts[selected_workout])
    else:
        print(f"No workout found with the name '{selected_workout}'.")

#-------------------#
# Read CSV File
#-------------------#
def load_exercise_database(csv_file_path):
    # Create an empty dictionary to hold all exercise categories
    exercise_database = {}

    # Open and read the CSV file
    with open(csv_file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Iterate through each row in the CSV file
        for row in reader:
            category = row['Category']
            exercise_name = row['Exercise Name']

            # Create a dictionary for this exercise
            exercise_data = {
                "description": row['Description'],
                "targeted_muscles": row['Targeted Muscles'].split(", "),  # Convert muscles to a list
                "equipment_needed": row['Equipment Needed'],
                "difficulty": row['Difficulty'],
                "repetitions": row['Repetitions']
            }

            # If the category doesn't exist in the database, create it
            if category not in exercise_database:
                exercise_database[category] = {}

            # Add the exercise to the corresponding category
            exercise_database[category][exercise_name] = exercise_data

    return exercise_database

#-------------------#
# Search Exercise Function
#-------------------#
def search_exercise():
    # Load the exercise database
    exercise_database = load_exercise_database("exercise_database.csv")

    # Ask the user for a keyword to search
    keyword = input("\nEnter a keyword to search for exercises (e.g., muscle group or exercise name): ").strip().lower()

    # Create a list to store matching exercises
    matching_exercises = []

    # Iterate over the exercise database to find matches
    for category, exercises in exercise_database.items():
        for exercise_name, details in exercises.items():
            # Check if the keyword is in the exercise name or targeted muscles
            if keyword in exercise_name.lower() or any(keyword in muscle.lower() for muscle in details['targeted_muscles']):
                matching_exercises.append({
                    "category": category,
                    "name": exercise_name,
                    "description": details['description'],
                    "targeted_muscles": details['targeted_muscles'],
                    "equipment_needed": details['equipment_needed'],
                    "difficulty": details['difficulty'],
                    "repetitions": details['repetitions']
                })

    # Display the results
    if matching_exercises:
        print("\nMatching Exercises:")
        for exercise in matching_exercises:
            print(f"""
            Category: {exercise['category']}
            Name: {exercise['name']}
            Description: {exercise['description']}
            Targeted Muscles: {', '.join(exercise['targeted_muscles'])}
            Equipment Needed: {exercise['equipment_needed']}
            Difficulty: {exercise['difficulty']}
            Repetitions: {exercise['repetitions']}
            """)
    else:
        print(f"\nNo exercises found for keyword: {keyword}")


#----------------------#
#
# App's Main Function
#
#----------------------#

def main():
    global saved_workouts
    saved_workouts = {}
    print("Welcome to the Fitness App!")


    while True:
        print("\nWhat would you like to do?")
        print("1. Browse Exercises")
        print("2. Build a Personalized Workout")
        print("3. View Saved Workouts")
        print("4. Exit")


        choice = input("Enter your choice: ").strip()

        if choice == "1":
            search_exercise()
        elif choice == "2":
           build_your_workout()
        elif choice == "3":
            display_saved_workout()
        elif choice == "4":
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")


main()



Saving exercise_database.csv to exercise_database.csv
Welcome to the Fitness App!

What would you like to do?
1. Browse Exercises
2. Build a Personalized Workout
3. View Saved Workouts
4. Exit
Enter your choice: 4
Goodbye!
